# Lung adenocarcinoma cancer detection pipeline


In [ ]:
#Libraries
from skimage import io
import os
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np 
%matplotlib inline
from PIL import Image

from sklearn.preprocessing import OneHotEncoder
from sklearn.utils.class_weight import compute_class_weight, compute_sample_weight

from keras.callbacks import EarlyStopping
import keras
from keras.models import *
from keras.layers import Input, Conv2D, MaxPooling2D, Conv2DTranspose, Dropout, concatenate, Reshape, Permute, Lambda, BatchNormalization
from keras.activations import softmax
from keras.optimizers import *
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, History
from keras import backend as K
from keras.utils import np_utils
from keras.preprocessing.image import ImageDataGenerator, DirectoryIterator
from keras_preprocessing import image

import glob 
import cv2
from scipy import misc
import sys
# import shutil

from zipfile import ZipFile 

from keras.layers import Lambda
from keras import backend as K

import imageio
import numpy as np
import random

import datetime
import json
from keras.models import model_from_json

import imgaug as ia
from imgaug import augmenters as iaa
import imageio

import pandas as pd
import six
import csv
 

ia.seed(1)

## 1. Classes

In [ ]:
"""
def load_patch(patch):
    p = io.imread(patch)
    p_array = np.asarray(p, dtype = "int32")
    return p_array 
    """

In [ ]:
# Mask generator into one-hot

patch_size = 512
batch_size = 16

class MasksBatchGenerator(DirectoryIterator):
    
    def __init__(self,
                directory, 
                image_data_generator,
                target_size = (patch_size, patch_size),
                num_channels = 1,
                classes=None, 
                class_mode=None,
                batch_size=batch_size, shuffle=True,
                seed=None,
                data_format=None,
                save_to_dir=None,
                save_prefix='',
                save_format='png',
                follow_links=False,
                subset=None,
                interpolation='nearest',
                preprocessing_function= None):

        super(MasksBatchGenerator,self).__init__(directory, 
                image_data_generator,                                              
                target_size=target_size,                      
                classes=classes,
                class_mode=class_mode,
                batch_size=batch_size,
                shuffle=shuffle,
                seed=seed,
                data_format=data_format,
                save_to_dir=save_to_dir,
                save_prefix=save_prefix,
                save_format=save_format,
                follow_links=follow_links,
                subset=subset,
                interpolation=interpolation)

        self.image_shape = (target_size[0],target_size[1],num_channels)
        self.preprocessing_function = preprocessing_function
        
    def _get_batches_of_transformed_samples(self, index_array):
        
        batch_x = np.zeros((len(index_array),) + self.image_shape, dtype = K.floatx())
        
        for i, j in enumerate(index_array):
            
            fname = self.filenames[j]

            img = cv2.cvtColor(cv2.imread(os.path.join(self.directory, fname)), cv2.COLOR_BGR2RGB)
            
            if self.preprocessing_function:
                img = self.preprocessing_function(img)
                
            batch_x[i] = img
            
        return batch_x

In [ ]:
# Patch generator

class PatchBatchGenerator(DirectoryIterator):
    
    def __init__(self,
                directory, 
                image_data_generator,
                target_size = (patch_size, patch_size),
                num_channels = 3,
                color_mode = "rgb",
                classes=None, 
                class_mode=None,
                batch_size=batch_size, shuffle=True,
                seed=None,
                data_format=None,
                save_to_dir=None,
                save_prefix='',
                save_format='jpg',
                follow_links=False,
                subset=None,
                interpolation='nearest',
                preprocessing_function= None):

        super(PatchBatchGenerator,self).__init__(directory, 
                image_data_generator,
                target_size=target_size,                      
                classes=classes,
                class_mode=class_mode,
                batch_size=batch_size,
                shuffle=shuffle,
                seed=seed,
                data_format=data_format,
                save_to_dir=save_to_dir,
                save_prefix=save_prefix,
                save_format=save_format,
                follow_links=follow_links,
                subset=subset,
                interpolation=interpolation)

        self.image_shape = (target_size[0],target_size[1],num_channels)
        
    def _get_batches_of_transformed_samples(self, index_array):
        
        batch_x = np.zeros((len(index_array),) + self.image_shape, dtype = K.floatx())
        
        for i, j in enumerate(index_array):
            
            fname = self.filenames[j]
            img = cv2.cvtColor(cv2.imread(os.path.join(self.directory, fname)), cv2.COLOR_BGR2RGB)        

            batch_x[i] = img/255.0
            
        return batch_x

In [ ]:
# Mask generator into one-hot with statial augmentation


class MasksBatchGeneratorAug(DirectoryIterator):
    
    def __init__(self,
                directory, 
                image_data_generator,
                target_size = (patch_size, patch_size),
                num_channels = 1,
                classes=None, 
                class_mode=None,
                batch_size=batch_size, shuffle=True,
                seed=None,
                data_format=None,
                save_to_dir=None,
                save_prefix='',
                save_format='png',
                follow_links=False,
                subset=None,
                interpolation='nearest',
                preprocessing_function= None,
                preprocessing_function_space= None):

        super(MasksBatchGeneratorAug,self).__init__(directory, 
                image_data_generator,
                target_size=target_size,                      
                classes=classes,
                class_mode=class_mode,
                batch_size=batch_size,
                shuffle=shuffle,
                seed=seed,
                data_format=data_format,
                save_to_dir=save_to_dir,
                save_prefix=save_prefix,
                save_format=save_format,
                follow_links=follow_links,
                subset=subset,
                interpolation=interpolation)

        self.image_shape = (target_size[0],target_size[1],num_channels)
        self.preprocessing_function_space = preprocessing_function_space
        self.preprocessing_function = preprocessing_function
        
    def _get_batches_of_transformed_samples(self, index_array):
        
        batch_x = np.zeros((len(index_array),) + self.image_shape, dtype = K.floatx())
        
        for i, j in enumerate(index_array):
            
            fname = self.filenames[j]
            img = cv2.cvtColor(cv2.imread(os.path.join(self.directory, fname)), cv2.COLOR_BGR2RGB)
            
            if self.preprocessing_function_space:
                img= self.preprocessing_function_space(img)
            
            if self.preprocessing_function:
                img = self.preprocessing_function(img)
                
            batch_x[i] = img
            
        return batch_x

In [ ]:
# Patch generator with statial augmentation


class PatchBatchGeneratorAug(DirectoryIterator):
    
    def __init__(self,
                directory, 
                image_data_generator,
                target_size = (patch_size, patch_size),
                num_channels = 3,
                color_mode = "rgb",
                classes=None, 
                class_mode=None,
                batch_size=batch_size, shuffle=True,
                seed=None,
                data_format=None,
                save_to_dir=None,
                save_prefix='',
                save_format='jpg',
                follow_links=False,
                subset=None,
                interpolation='nearest',
                preprocessing_function_space= None,
                preprocessing_function= None):

        super(PatchBatchGeneratorAug,self).__init__(directory, 
                image_data_generator,
                target_size=target_size,                      
                classes=classes,
                class_mode=class_mode,
                batch_size=batch_size,
                shuffle=shuffle,
                seed=seed,
                data_format=data_format,
                save_to_dir=save_to_dir,
                save_prefix=save_prefix,
                save_format=save_format,
                follow_links=follow_links,
                subset=subset,
                interpolation=interpolation)

        self.image_shape = (target_size[0],target_size[1],num_channels)
        self.preprocessing_function_space = preprocessing_function_space
        
    def _get_batches_of_transformed_samples(self, index_array):
        
        batch_x = np.zeros((len(index_array),) + self.image_shape, dtype = K.floatx())
        
        for i, j in enumerate(index_array):
            
            fname = self.filenames[j]

            img = cv2.cvtColor(cv2.imread(os.path.join(self.directory, fname)), cv2.COLOR_BGR2RGB)
            
            if self.preprocessing_function_space:
                img= self.preprocessing_function_space(img)
                
            batch_x[i] = img/255.0
            
        return batch_x

In [ ]:
# Patch generator with spatial and brightness augmentation

class PatchBatchGeneratorAugCol(DirectoryIterator):
    
    def __init__(self,
                directory, 
                image_data_generator,
                target_size = (patch_size, patch_size),
                num_channels = 3,
                color_mode = "rgb",
                classes=None, 
                class_mode=None,
                batch_size=batch_size, shuffle=True,
                seed=None,
                data_format=None,
                save_to_dir=None,
                save_prefix='',
                save_format='jpg',
                follow_links=False,
                subset=None,
                interpolation='nearest',
                preprocessing_function_colour = None,
                preprocessing_function_space= None):

        super(PatchBatchGeneratorAugCol,self).__init__(directory, 
                image_data_generator,
                target_size=target_size,                      
                classes=classes,
                class_mode=class_mode,
                batch_size=batch_size,
                shuffle=shuffle,
                seed=seed,
                data_format=data_format,
                save_to_dir=save_to_dir,
                save_prefix=save_prefix,
                save_format=save_format,
                follow_links=follow_links,
                subset=subset,
                interpolation=interpolation)

        self.image_shape = (target_size[0],target_size[1],num_channels)
        self.preprocessing_function_space = preprocessing_function_space
        self.preprocessing_function_colour = preprocessing_function_colour
        
    def _get_batches_of_transformed_samples(self, index_array):
        
        batch_x = np.zeros((len(index_array),) + self.image_shape, dtype = K.floatx())
        
        for i, j in enumerate(index_array):
            
            fname = self.filenames[j]

            img = cv2.cvtColor(cv2.imread(os.path.join(self.directory, fname)), cv2.COLOR_BGR2RGB)
            
            if self.preprocessing_function_space:
                img= self.preprocessing_function_space(img)
                
            if self.preprocessing_function_colour:
                img= self.preprocessing_function_colour(img)
                
            batch_x[i] = img/255.0
            
        return batch_x

## 2. Functions

In [ ]:
# Open image function

def open_image(path):
  newImage = Image.open(path)
  return newImage

In [ ]:
# Convertor from masks with many classes(colours) to one-hot vector with only 3 classes

## 0 is background, 1 is healthy, 2 is cancer
def rgb_to_onehot_3cls(rgb_arr):
    
    color_dict = {0: [255, 255, 255], # white- background  bgr rgb GOOD
              1: [255, 0, 255], # magenta - normal 
              2: [102, 102, 102], # gray- other GOOD
              3: [50, 160, 50], # also something green- acinary tumour
              4: [0, 255, 0], # green lime- lepidic tumour GOOD
              5: [0, 0, 0], # black- solid tumour GOOD
              6: [0, 75, 0], # - darker gree tumour
              7: [130, 170, 130], # light green- micropapollary tumour 
                 }          
             
    num_classes = len(color_dict)-5
    shape = rgb_arr.shape[:2]
    arr = np.zeros( shape, dtype=np.int8 )
    
    for i, cls in color_dict.items():
        
        if i == 0:
            
            mask0 = rgb_arr[:, :, 0] == cls[0]
            mask1 = rgb_arr[:, :, 1] == cls[1]
            mask2 = rgb_arr[:, :, 2] == cls[2]
            mask = np.logical_and(np.logical_and(mask0, mask1), mask2)
            arr[mask] = i
            
        elif i == 1 or i == 2:
            
            mask0 = rgb_arr[:, :, 0] == cls[0]
            mask1 = rgb_arr[:, :, 1] == cls[1]
            mask2 = rgb_arr[:, :, 2] == cls[2]
            mask = np.logical_and(np.logical_and(mask0, mask1), mask2)
            arr[mask] = 1
            
        elif i == 3 or i == 4 or i == 5 or i == 6 or i == 7:
            
            mask0 = rgb_arr[:, :, 0] == cls[0]
            mask1 = rgb_arr[:, :, 1] == cls[1]
            mask2 = rgb_arr[:, :, 2] == cls[2]
            mask = np.logical_and(np.logical_and(mask0, mask1), mask2)
            arr[mask] = 2
        
    arr_vec=np.zeros((arr.shape[0], arr.shape[1], num_classes))
    
    for i in range(num_classes):
        
        mask = (arr == i)
        arr_vec[:, :, i][mask] = 1 
        
    return arr_vec

In [ ]:
# Convertor from one-hot to RGB image with 3 colours

def onehot_to_rgb_3cls(onehot):
    
    color_dict = {0: [255, 255, 255], # white- background  bgr rgb
              1: [255, 0, 255], # non-cancer -blue
              2: [0, 0, 0], # cancer -red
                 }
    single_layer = np.argmax(onehot, axis=-1)
    output = np.zeros( onehot.shape[:2]+(3,) )
    for k in color_dict.keys():
        output[single_layer==k] = color_dict[k]
    return np.uint8(output)

In [ ]:
# Brightness augmentation function

def brightness_augmentation(factor):
    factor = factor
    
    def augment(img): 

        hsv = cv2.cvtColor(img, cv2.COLOR_RGB2HSV) #convert to hsv
        hsv = np.array(hsv, dtype=np.float64)
        hsv[:, :, 2] = hsv[:, :, 2] * (factor + np.random.uniform()) #scale channel V uniformly
        hsv[:, :, 2][hsv[:, :, 2] > 255] = 255 #reset out of range values
        rgb = cv2.cvtColor(np.array(hsv, dtype=np.uint8), cv2.COLOR_HSV2RGB)     
        augment = rgb
        return augment
    
    return augment 

In [ ]:
# Weighted categorical cross-entropy function

def weighted_categorical_crossentropy(weights):
    weights = K.variable(weights)
        
    def loss(y_true, y_pred):
        
        # scale predictions so that the class probas of each sample sum to 1
        y_pred /= K.sum(y_pred, axis=-1, keepdims=True)
        # clip to prevent NaN's and Inf's
        y_pred = K.clip(y_pred, K.epsilon(), 1 - K.epsilon())
        # calc
        loss = y_true * K.log(y_pred) * weights
        loss = -K.sum(loss, -1)
        
        return loss
    
    return loss

In [ ]:
# Score measurements

def precision(y_true, y_pred):

    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def recall(y_true, y_pred):

    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def fbeta_score(y_true, y_pred, beta=1):

    if beta < 0:
        raise ValueError('The lowest choosable beta is zero (only precision).')
        
    # If there are no true positives, fix the F score at 0 like sklearn.
    if K.sum(K.round(K.clip(y_true, 0, 1))) == 0:
        return 0

    p = precision(y_true, y_pred)
    r = recall(y_true, y_pred)
    bb = beta ** 2
    fbeta_score = (1 + bb) * (p * r) / (bb * p + r + K.epsilon())
    return fbeta_score

def fmeasure(y_true, y_pred):

    return fbeta_score(y_true, y_pred, beta=1)


In [ ]:
# Metrics visualisation function

def render_table(data, col_width = 3.0, row_height = 0.625, font_size = 14,
                     header_color = '#F8F8FF', row_colors = ['w'], edge_color = '#D3D3D3',
                     bbox = [0, 0, 1, 1], header_columns = 0,
                     ax = None, **kwargs):
    if ax is None:
        size = (np.array(data.shape[::-1]) + np.array([0, 1])) * np.array([col_width, row_height])
        fig, ax = plt.subplots(figsize=size)
        ax.axis('off')

    mpl_table = ax.table(cellText = data.values, bbox = bbox, colLabels = data.columns, **kwargs)

    mpl_table.auto_set_font_size(False)
    mpl_table.set_fontsize(font_size)

    for k, cell in six.iteritems(mpl_table._cells):
        cell.set_edgecolor(edge_color)
        if k[0] == 0 or k[1] < header_columns:
            cell.set_text_props(weight = 'bold', color = '#000000')
            cell.set_facecolor(header_color)
        else:
            cell.set_facecolor(row_colors[k[0]%len(row_colors)])
    return ax

## Model implementation

In [ ]:
#### Model parameters (inputs)

n_classes = 3 
patch_size = 512 #both height and width 

batch_size = 16
n_epoch = 100

learning_rate = 0.01
patience = 30

# Weights
class_weights = [1,1,2]
type_loss_function = weighted_categorical_crossentropy(class_weights)
# type_loss_function = keras.losses.categorical_crossentropy

# Spatial augmentation function
seq = iaa.Sometimes(0.75,iaa.SomeOf((1,3),[iaa.Affine(rotate=(0, 360),cval=(255)), iaa.Fliplr(0.5), iaa.Flipud(0.5)], random_order=True))
seq_det = seq.to_deterministic()

# Colour augmentation

colour_shift = 0.0500
colour_augmentation_function = brightness_augmentation(colour_shift)

# Set time experiment 

current_date = datetime.datetime.today().strftime('%Y-%m-%d')

# Experiment order

Exp = '_Exp_512'

# Directories

history_path = os.path.dirname(os.path.realpath("Train")) + '/History_data_'
weights_path = os.path.dirname(os.path.realpath("Train")) + '/Weights_data_'



In [ ]:
# Data directory
Test_path = "/home/guest/DL/Teodora/Project/Test512"
Train_path = "/home/guest/DL/Teodora/Project/Train512"
Val_path = "/home/guest/DL/Teodora/Project/Validation512"

In [ ]:
#U-net implementation

inputs = Input((patch_size, patch_size, 3))
conv1 = Conv2D(32, (3, 3), activation='relu', padding='same', dilation_rate = 2)(inputs)
BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001, center=True, scale=True, beta_initializer='zeros', gamma_initializer='ones', moving_mean_initializer='zeros', moving_variance_initializer='ones', beta_regularizer=None, gamma_regularizer=None, beta_constraint=None, gamma_constraint=None)
conv1 = Conv2D(32, (3, 3), activation='relu', padding='same', dilation_rate = 2)(conv1)
BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001, center=True, scale=True, beta_initializer='zeros', gamma_initializer='ones', moving_mean_initializer='zeros', moving_variance_initializer='ones', beta_regularizer=None, gamma_regularizer=None, beta_constraint=None, gamma_constraint=None)
pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)

print(conv1.shape)

conv2 = Conv2D(64, (3, 3), activation='relu', padding='same')(pool1)
BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001, center=True, scale=True, beta_initializer='zeros', gamma_initializer='ones', moving_mean_initializer='zeros', moving_variance_initializer='ones', beta_regularizer=None, gamma_regularizer=None, beta_constraint=None, gamma_constraint=None)
conv2 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv2)
BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001, center=True, scale=True, beta_initializer='zeros', gamma_initializer='ones', moving_mean_initializer='zeros', moving_variance_initializer='ones', beta_regularizer=None, gamma_regularizer=None, beta_constraint=None, gamma_constraint=None)
pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)


print(conv2.shape)

conv3 = Conv2D(128, (3, 3), activation='relu', padding='same')(pool2)
BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001, center=True, scale=True, beta_initializer='zeros', gamma_initializer='ones', moving_mean_initializer='zeros', moving_variance_initializer='ones', beta_regularizer=None, gamma_regularizer=None, beta_constraint=None, gamma_constraint=None)
conv3 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv3)
BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001, center=True, scale=True, beta_initializer='zeros', gamma_initializer='ones', moving_mean_initializer='zeros', moving_variance_initializer='ones', beta_regularizer=None, gamma_regularizer=None, beta_constraint=None, gamma_constraint=None)
pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)

print(conv3.shape)

conv4 = Conv2D(256, (3, 3), activation='relu', padding='same')(pool3)
BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001, center=True, scale=True, beta_initializer='zeros', gamma_initializer='ones', moving_mean_initializer='zeros', moving_variance_initializer='ones', beta_regularizer=None, gamma_regularizer=None, beta_constraint=None, gamma_constraint=None)
conv4 = Conv2D(256, (3, 3), activation='relu', padding='same')(conv4)
BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001, center=True, scale=True, beta_initializer='zeros', gamma_initializer='ones', moving_mean_initializer='zeros', moving_variance_initializer='ones', beta_regularizer=None, gamma_regularizer=None, beta_constraint=None, gamma_constraint=None)
pool4 = MaxPooling2D(pool_size=(2, 2))(conv4)

print(conv4.shape)

conv5 = Conv2D(512, (3, 3), activation='relu', padding='same')(pool4)
BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001, center=True, scale=True, beta_initializer='zeros', gamma_initializer='ones', moving_mean_initializer='zeros', moving_variance_initializer='ones', beta_regularizer=None, gamma_regularizer=None, beta_constraint=None, gamma_constraint=None)
conv5 = Conv2D(512, (3, 3), activation='relu', padding='same')(conv5)
BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001, center=True, scale=True, beta_initializer='zeros', gamma_initializer='ones', moving_mean_initializer='zeros', moving_variance_initializer='ones', beta_regularizer=None, gamma_regularizer=None, beta_constraint=None, gamma_constraint=None)
print(conv5.shape)

up6 = concatenate([Conv2DTranspose(256, (2, 2), strides=(2, 2), padding='same')(conv5), conv4], axis=3)
conv6 = Conv2D(256, (3, 3), activation='relu', padding='same')(up6)
BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001, center=True, scale=True, beta_initializer='zeros', gamma_initializer='ones', moving_mean_initializer='zeros', moving_variance_initializer='ones', beta_regularizer=None, gamma_regularizer=None, beta_constraint=None, gamma_constraint=None)
conv6 = Conv2D(256, (3, 3), activation='relu', padding='same')(conv6)
BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001, center=True, scale=True, beta_initializer='zeros', gamma_initializer='ones', moving_mean_initializer='zeros', moving_variance_initializer='ones', beta_regularizer=None, gamma_regularizer=None, beta_constraint=None, gamma_constraint=None)

up7 = concatenate([Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(conv6), conv3], axis=3)
conv7 = Conv2D(128, (3, 3), activation='relu', padding='same')(up7)
BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001, center=True, scale=True, beta_initializer='zeros', gamma_initializer='ones', moving_mean_initializer='zeros', moving_variance_initializer='ones', beta_regularizer=None, gamma_regularizer=None, beta_constraint=None, gamma_constraint=None)
conv7 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv7)
BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001, center=True, scale=True, beta_initializer='zeros', gamma_initializer='ones', moving_mean_initializer='zeros', moving_variance_initializer='ones', beta_regularizer=None, gamma_regularizer=None, beta_constraint=None, gamma_constraint=None)

up8 = concatenate([Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(conv7), conv2], axis=3)
conv8 = Conv2D(64, (3, 3), activation='relu', padding='same')(up8)
BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001, center=True, scale=True, beta_initializer='zeros', gamma_initializer='ones', moving_mean_initializer='zeros', moving_variance_initializer='ones', beta_regularizer=None, gamma_regularizer=None, beta_constraint=None, gamma_constraint=None)
conv8 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv8)
BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001, center=True, scale=True, beta_initializer='zeros', gamma_initializer='ones', moving_mean_initializer='zeros', moving_variance_initializer='ones', beta_regularizer=None, gamma_regularizer=None, beta_constraint=None, gamma_constraint=None)

up9 = concatenate([Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same')(conv8), conv1], axis=3)
conv9 = Conv2D(32, (3, 3), activation='relu', padding='same')(up9)
BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001, center=True, scale=True, beta_initializer='zeros', gamma_initializer='ones', moving_mean_initializer='zeros', moving_variance_initializer='ones', beta_regularizer=None, gamma_regularizer=None, beta_constraint=None, gamma_constraint=None)
conv10 = Conv2D(32, (3, 3), activation='relu', padding='same')(conv9)
BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001, center=True, scale=True, beta_initializer='zeros', gamma_initializer='ones', moving_mean_initializer='zeros', moving_variance_initializer='ones', beta_regularizer=None, gamma_regularizer=None, beta_constraint=None, gamma_constraint=None)

conv11 = Conv2D(n_classes, (1, 1), activation='linear')(conv10)
print(conv10.shape)
out=conv11
out = Lambda(lambda x: softmax(x, axis = 3))(conv11)

model = Model(inputs=[inputs], outputs=[out])

keras.optimizers.Adam(lr=learning_rate, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)#learning rate optimisation
# keras.optimizers.Adadelta(lr=learning_rate , rho=0.95, epsilon=None, decay=0.0)
model.compile(optimizer = 'adam', loss = type_loss_function, metrics=['accuracy', recall, precision, fmeasure])

model.summary()

## Train data 

In [ ]:
# Augmentation parameters

patch_arg = dict()
mask_arg = dict()



train_patch_generator = PatchBatchGeneratorAugCol(
                                            **patch_arg,
                                            directory = Train_path + '/Patches',
                                            image_data_generator = ImageDataGenerator,
                                            target_size = (patch_size, patch_size),                                  
                                            color_mode = "rgb",
                                            num_channels = 3,
                                            batch_size = batch_size,
                                            class_mode = None,
                                            shuffle = True,
                                            preprocessing_function_space = seq_det.augment_image,
                                            preprocessing_function_colour = colour_augmentation_function,
                                            seed = 1)
    
train_mask_generator = MasksBatchGeneratorAug(  
                                            **mask_arg,
                                            directory = Train_path + '/Masks',
                                            image_data_generator = ImageDataGenerator,
                                            target_size = (patch_size, patch_size),
                                            num_channels = n_classes,
                                            batch_size = batch_size,
                                            class_mode = None,
                                            shuffle = True,                                      
                                            seed = 1,
                                            preprocessing_function_space = seq_det.augment_image,
                                            preprocessing_function = rgb_to_onehot_3cls)

print('Train sets ready')

train_patch_generator_copy = train_patch_generator
train_mask_generator_copy = train_mask_generator

train_generator = zip(train_patch_generator, train_mask_generator)

# #Some check-ups
# a= next(train_generator)
# [a1,b1] = list(a) 
# (f, [ax0,ax1]) = plt.subplots(1, 2, figsize = (24, 6))
# ax0.imshow(a1[1])
# ax1.imshow(b1[1])
# print (train_patch_generator.__getitem__(0).shape)
# print (train_mask_generator.__getitem__(0).shape)



In [ ]:
n_train_samples = len([name for name in os.listdir(Train_path + '/Patches/Group1/')])
print (n_train_samples)

## Validation data

In [ ]:
patch_arg = dict()
mask_arg = dict()

val_patch_generator = PatchBatchGenerator(
                                            directory = Val_path + '/Patches',
                                            image_data_generator = ImageDataGenerator,
                                            target_size = (patch_size, patch_size),
                                            color_mode = "rgb",
                                            batch_size = batch_size,
                                            class_mode = None,
                                            shuffle = False,
                                            seed= 1)
    
val_mask_generator = MasksBatchGenerator(
                                            directory = Val_path + '/Masks',
                                            image_data_generator = ImageDataGenerator,
                                            target_size = (patch_size, patch_size),
                                            batch_size = batch_size,
                                            num_channels = n_classes,
                                            class_mode = None,
                                            shuffle = False,
                                            seed = 1,
                                            preprocessing_function = rgb_to_onehot_3cls)

print('Validation sets ready')
validation_generator = zip(val_patch_generator, val_mask_generator)


In [ ]:
n_validation_samples  = len([name for name in os.listdir(Val_path + '/Patches/Group1/')])
print (n_validation_samples)

## Weight, history and model run

In [ ]:
# Weights Part 1

if not os.path.exists(weights_path):
        os.makedirs(weights_path)

weights_file = weights_path + '/Weights_' + current_date + Exp +'_{epoch:02d}.h5'

if not os.path.exists(weights_file):
    with open(weights_file, 'w'): pass

In [ ]:
model_checkpoint = ModelCheckpoint(weights_file, monitor='val_loss', verbose=0, save_best_only=True, save_weights_only=False, mode='auto')


earlystop = EarlyStopping(monitor = 'val_loss', min_delta = 0, patience = patience, verbose = 1, mode = 'auto')
#history = keras.callbacks.History()


history = model.fit_generator(
                    train_generator,
                    steps_per_epoch = n_train_samples/ batch_size,
                    epochs = n_epoch,
                    validation_data = validation_generator,
                    shuffle = True,
                    callbacks = [model_checkpoint, earlystop],
                    validation_steps = n_validation_samples/batch_size,
                    #     use_multiprocessing=True,
                    #     workers=2
)

In [ ]:
# Weights Part 2

model.save_weights(weights_file)

print("Weights saved")

In [ ]:
# History

if not os.path.exists(history_path):
        os.makedirs(history_path)

history_file = history_path + '/History_data_' + current_date + Exp +'.json'

if not os.path.exists(history_file):
    with open(history_file, 'w'): pass

with open (history_file,'w') as file:
    json.dump(history.history, file)
    
print("History saved")  

In [ ]:
print(history.history.keys())

(f, [ax0, ax1, ax2, ax3]) = plt.subplots(1, 4, figsize = (24, 6))

ax0.plot(history.history['loss'])
ax0.plot(history.history['val_loss'])
ax0.set_title('model loss')
ax0.set_ylabel('loss')
ax0.set_xlabel('epoch')
ax0.legend(['train', 'valid'], loc = 'upper left')
ax0.axis([0, n_epoch, 0, max(history.history['val_loss'])])

ax1.plot(history.history['acc'])
ax1.plot(history.history['val_acc'])
ax1.set_title('model accuracy')
ax1.set_ylabel('accuracy')
ax1.set_xlabel('epoch')
ax1.legend(['train', 'valid'], loc = 'upper left')
ax1.axis([0, n_epoch, 0, 1])

ax2.plot(history.history['recall'])
ax2.plot(history.history['val_recall'])
ax2.set_title('model recall')
ax2.set_ylabel('recall')
ax2.set_xlabel('epoch')
ax2.legend(['train', 'valid'], loc = 'upper left')
ax2.axis([0, n_epoch, 0, 1])

ax3.plot(history.history['precision'])
ax3.plot(history.history['val_precision'])
ax3.set_title('model precision')
ax3.set_ylabel('precision')
ax3.set_xlabel('epoch')
ax3.legend(['train', 'valid'], loc = 'upper left')
ax3.axis([0, n_epoch, 0, 1])

plt.tight_layout()
plt.show()

In [ ]:
%reset -f